<a href="https://colab.research.google.com/github/fbeilstein/dbms/blob/master/DB_lecture_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Anti-Entropy and Dissemination**

Most of the communication patterns we’ve been discussing so far were either peer-to-peer or one-to-many (coordinator and replicas). To reliably propagate data records throughout the system, we need the propagating node to be available and able to reach the other nodes, but even then the throughput is limited to a single machine.


Quick and reliable propagation may be less applicable to data records and more
important for the cluster-wide metadata, such as membership information (joining
and leaving nodes), node states, failures, schema changes, etc. Messages containing this information are generally infrequent and small, but have to be propagated as quickly and reliably as possible. Such updates can generally be propagated to all nodes in the cluster using one of the three broad groups of approaches:

* Notification broadcast from one process to all others.
* Periodic peer-to-peer information exchange. Peers connect pairwise and
exchange messages.
* Cooperative broadcast, where message recipients become broadcasters and
help to spread the information quicker and more reliably.


**Broadcasting** 
- (+) the most straightforward approach
- (+) works well for small number of nodes
- (-) in large clusters it can get expensive
- (-) overdependence on a single process
- (-) individual processes may not always know about the existence of all other processes
- (-) must be some overlap in time during which both the broadcasting process and each one of its recipients are up

**relax constraints** (updates may fail to propagate) + **anti-entropy** (bring nodes back in sync if failure) <=> **primary delivery** + **periodic sync**


Entropy is a property that represents the measure of disorder in the system. In a distributed system, **entropy** represents a degree of state divergence between the nodes. Since this property is undesired and its amount should be kept to a minimum.

**Anti-entropy** is used to lower the convergence time bounds in eventually consistent systems. Simple pairwise comparison is too costly, think of something better.

**Read Repair**


It is easiest to detect divergence between the replicas during the read, since at that point we can contact replicas, request the queried state from each one of them, and see whether or not their responses match. Note that in this case we do not query an entire dataset stored on each replica, and we limit our goal to just the data that was requested by the client.

**read repair**
- coordinator <-read- replica 1,2,3,4,...
- coordinator compares responces
- coordinator -update-> replica 2,4

Contactieng all replicas -- too long. Dynamo-style solution: contact only the number of nodes that satisfies the chosen consistency level. If we do quorum reads and writes, we still get consistent results, but some of the replicas still might not contain all the writes.


Read repair 
* **blocking**  the original client request has to wait until the coordinator  “repairs” the replicas. For quorum reads ensures read monotonicity (see Session Models). 
* **asynchronous** simply schedules a task that can be executed after results are returned to the user.



To detect exactly which records differ between replica responses, some databases (Apache Cassandra) use specialized iterators with merge listeners, which reconstruct differences between the merged result and individual inputs.

**Digest Reads**

Problem: reading full data too long.

- coordinator -normal read-> node 1
- coordinator -digest request-> nodes 2,3,4,5,...
- coordinator <-data- node 1
- coordinator <-hashes (non-cryptographic)- nodes 2,3,4,5,...
- coordinator checks hashes and makes decision if the full-read is neaded.


Digests are usually computed using a noncryptographic hash function, such as MD5, since it has to be computed quickly to make the “happy path” performant. Hash functions can have collisions, but their probability is negligible for most real-world systems. Since databases often use more than just one anti-entropy mechanism, we can expect that, even in the unlikely event of a hash collision, data will be reconciled by the different subsystem.

**Hinted Handoff**


Another anti-entropy approach is called hinted handoff, a write-side repair mechanism. As an example see [hinted hadoff in Cassandra](https://docs.datastax.com/en/cassandra-oss/2.1/cassandra/dml/dml_about_hh_c.html). If the target node fails to acknowledge the write, the write coordinator or one of the replicas stores a special record, called a hint, which is replayed to the target node as soon as it comes back up.


In Apache Cassandra, unless the ANY consistency level is in use, hinted
writes aren’t counted toward the replication factor, since the data in the hint log isn’t accessible for reads and is only used to help the lagging participants catch up.


Some databases, for example Riak, use sloppy quorums together with hinted handoff. With sloppy quorums, in case of replica failures, write operations can use additional healthy nodes from the node list, and these nodes do not have to be target replicas for the executed operations.

- -write->A,B,C but B is down
- -write->A,C, -store hint->D
- B is up: D-forward hint->B


Under similar circumstances, if nodes B, C are briefly separated from the rest of the cluster by the network partition, and a sloppy quorum write was done against A,D,E, a read on B,C, immediately following this write, would not observe the latest read. In other words, sloppy quorums improve availability at the cost of consistency

**Merkle Trees**

Git, Dynamo DB, Cassandra, IPFS, bitcoin (tell about bitcoin, that should be interesting)

Since read repair can only fix inconsistencies on the currently queried data, we
should use different mechanisms to find and repair inconsistencies in the data that is not actively queried.


Finding exactly which rows have diverged between the replicas requires exchanging and comparing the data records pairwise -> too long,too heavy. 

![img](https://upload.wikimedia.org/wikipedia/commons/6/6d/Hash_tree.png)

* The lowest level of this hash tree is built by scanning an entire table
holding data records, and computing hashes of record ranges. 
* Higher tree levels contain hashes of the lower-level hashes, building a hierarchical representation that allows us to quickly detect inconsistencies by comparing the hashes. 


**usage**
* To determine whether or not there’s an inconsistency between the two replicas, we only need to compare the root-level hashes from their Merkle trees. 
* By comparing hashes pairwise from top to bottom, it is possible to locate ranges holding differences between the nodes, and repair data records contained in them.


(-) Since Merkle trees are calculated recursively from the bottom to the top, a change in data triggers recomputation of the entire subtree. 


(-) A trade-off between the size of a tree (consequently, sizes of exchanged messages) and its precision (how small and exact data ranges are)

**Bitmap Version Vectors**


More recent research on this subject introduces bitmap version vectors ([article](https://haslab.uminho.pt/tome/files/global_logical_clocks.pdf), [explanation](https://hazm.at/mox/distributed-system/algorithm/consistency/version-vector/index.html)) can be used to resolve data conflicts based on recency: each node keeps a per-peer log of operations that have occurred locally or were replicated. 


To track replica states, we use node-local logical clocks. Each clock represents a set of dots, representing writes this node has seen directly (coordinated by the node itself), or transitively (coordinated by and replicated from the other nodes).

During anti-entropy, logs are compared, and missing data is replicated to the target node. Each write, coordinated by a node, is represented by a dot (i,n): an event with a node-local sequence number i coordinated by the node n. The sequence number i starts with 1 and is incremented each time the node executes a write operation.

![img](https://hazm.at/mox/distributed-system/algorithm/consistency/version-vector/bitmapped-version-vector.png)


* In the node logical clock, events coordinated by the node itself will have no gaps. 
* If some writes aren’t replicated from the other nodes, the clock will contain gaps. 
* To get two nodes back in sync, they can exchange logical clocks, identify gaps represented by the missing dots, and then replicate data records associated with them. 
* As soon as all the nodes in the system have seen consecutive values up to
the index i, the version vector can be truncated up to this index.



A possible downside is that, if the node was down for an extended time
period, peer nodes can’t truncate the log, since data still has to be replicated to the lagging node once it comes back up.

**Gossip Dissemination**



To involve other nodes, and propagate updates with the **reach** of a broadcast and the **reliability** of anti-entropy, we can use gossip protocols.


Gossip protocols are probabilistic communication procedures based on how rumors
are spread in human society or how diseases propagate in the population. 

The main objective of gossip protocols is to use cooperative propagation to disseminate information from one process to the rest of the cluster. 

* process holds a record that has to be spread around sends message to random peers
* process that hasn’t received the update yet can obtain it and become a new holder
* As soon as the holder processes become certain that the update was propagated, they move to the removed state and stop sending messages


* Since gossip protocols generally do not require explicit coordination, they can be useful in systems with flexible membership (where nodes are joining and
leaving frequently) or mesh networks.
* Gossip protocols are very robust and help to achieve high reliability in the presence of failures inherent to distributed systems. Since messages are relayed in a randomized manner, they still can be delivered even if some communication components between them fail, just through the different paths. 

**Gossip Mechanics**


Processes periodically select $f$ peers at random (**fanout**, configurable) and exchange currently “hot” information with them. Whenever the process learns about a new piece of information from its peers, it will attempt to pass it on further. 


Because peers are selected probabilistically, there will always be some overlap, and messages will get delivered repeatedly and may continue circulating for some time. 


The amount of time the system requires to reach convergence is called **latency**.
There’s a slight difference between reaching convergence (stopping the gossip process) and delivering the message to all peers, since there might be a short period during which all peers are notified, but gossip continues. 


fanout $\downarrow$ -> latency $\uparrow$ 

fanout $\uparrow$ -> latency $\downarrow$ 

Interest loss can be computed 
* probabilistically (the probability of propagation stop is computed for each process on every step) 
* using a threshold (the number of received duplicates is counted, and propagation is stopped when this number is too high).

In terms of consistency, gossip protocols offer convergent consistency:
nodes have a higher probability to have the same view of the events that occurred further in the past.

**Overlay Networks**


Even though gossip protocols are important and useful, they’re usually applied for a narrow set of problems. Nonepidemic approaches can distribute the message with nonprobabilistic certainty, less redundancy, and generally in a more optimal way. 

Gossip algorithms are often praised for their scalability and the fact it
is possible to distribute a message within log N message rounds (where N is the size of the cluster), but it’s important to keep the number of redundant
messages generated during gossip rounds in mind as well. 



Selecting nodes at random greatly improves system robustness: if there is a network partition, messages will be delivered eventually if there are links that indirectly connect two processes. (-) it is not message optimal: to guarantee robustness, we have to maintain redundant connections between the peers and send redundant messages.


* **temporary fixed topology** Create an overlay network of peers: nodes can sample their peers and select the best contact points based on proximity (usually measured by the latency). 
* **spanning trees** Having such a graph, messages can be distributed in a fixed number of steps. (-) it might lead to forming interconnected “islands” of peers having strong preferences toward each other.


To keep the number of messages low, while allowing quick recovery in case of a connectivity loss, we can mix both approaches — fixed topologies and tree-based broadcast—when the system is in a stable state, and fall back to gossip for failover and system recovery

**Hybrid Gossip**


**Push/lazy-push multicast trees** (Plumtrees) make a trade-off between epidemic and tree-based broadcast primitives. Plumtrees work by creating a spanning tree overlay of nodes to actively distribute messages with the smallest overhead.

* Under normal conditions, nodes send full messages to just a small subset of peers provided by the peer sampling service.
    - Each node sends the full message to the small subset of nodes
    - lazily forwards only the message ID for the rest
    - if the node receives the identifier of a message it has never seen, it can query its peers to get it. 
* In case of failures, protocol falls back to the gossip approach through lazy-push steps, broadcasting the message and repairing the overlay.


Due to the nature of distributed systems, any node or link between the nodes might fail at any time, making it impossible to traverse the tree when the segment becomes unreachable. The lazy gossip network helps to notify peers about seen messages in order to construct and repair the tree.


One of the advantages of using the lazy-push mechanism for tree construction and
repair is that in a network with constant load, it will tend to generate a tree that also minimizes message latency, since nodes that are first to respond are added to the broadcast tree

**Partial Views**


Broadcasting messages to all known peers and maintaining a full view of the cluster can get expensive and impractical, especially if the churn (measure of the number of joining and leaving nodes in the system) is high. 

To avoid this, gossip protocols often use a peer sampling service. This service maintains a partial view of the cluster, which is periodically refreshed using gossip. Partial views overlap, as some degree of redundancy is desired in gossip protocols, but too much redundancy means we’re doing extra work.


**Hybrid Partial View** (HyParView) protocol maintains a small active view and a larger passive view of the cluster. 
* Nodes from the active view create an overlay that can be used for dissemination. 
* Passive view is used to maintain a list of nodes that can be used to replace the failed ones from the active view.

Periodically, nodes perform a shuffle operation, during which they exchange their active and passive views. During this exchange, nodes add the members from both passive and active views they receive from their peers to their passive views, cycling out the oldest values to cap the list size.

The active view is updated depending on the state changes of nodes in this view and requests from peers. 

A | B | C
---|---|---
active: B, replacement: C | |
req to B| fails | 
req to C| | OK
try add C to active | | if active list not full -> accept
 | | if active list full -> try decline
active list not empty -> accept decline | | decline
active list empty -> reject decline | | replace one of actives for A

This helps bootstrapping or recovering nodes to quickly become effective members of the cluster at the cost of cycling some connections.


One of the performance and quality measures is how quickly a peer sampling service converges to a stable overlay in cases of topology reorganization. HyParView scores rather high here, because of how the views are maintained and since it gives priority to bootstrapping processes.